In [1]:
!pip install ultralytics -q
import torch
import os
import shutil
import glob
from ultralytics import YOLO
import ultralytics.nn.tasks

print(f"GPU ACTIVE: {torch.cuda.get_device_name(0)}")

if os.path.exists("Research Results"):
    shutil.rmtree("Research_results")
os.makedirs("Research_Results")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.5 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
GPU ACTIVE: Tesla P100-PCIE-16GB


In [2]:
import torch.nn as nn
import torchvision

#Bhaskar Net first

#dialated convolution

class DialatedConv(nn.Module):
    def __init__(self, inc, outc, k=3, s=1, p=None, g=1, d=1, act=True, bn=True, bias=False):
        super().__init__()
        if p is None: p = d
        self.conv = nn.Conv2d(inc, outc, k, s, p, dialation=d, group=g,bias=bias)
        self.bn = nn.BatchNorm2d(outc) if bn else nn.Identity()
        self.act = nn.SiLU() if act else nn.Identify()
    def forward(self, x):
        return self.act(self.bn(self.conv(x)))
#pseduo 3d

class Scalseq_Fast(nn.Module):
    def __init__(self, *args):
        super().__init__()
        flat_args = []
        for a in args:
            if isinstance(a, (list, tuple)): flat_args.extend(a)
            else: flat_args.append(a)
        c_p3, c_p4, c_p5 = int(flat_args[-3], int(flat_args[-2]), int(flat_args[-1]))
        self.c_p3, self.c_p4, self.c_p5 = c_p3, c_p4, c_p5
        out = c_p3
        self.cv3 = nn.Conv2d(c_p3, out, 1)
        self.cv4 = nn.Conv2d(c_p4, out, 1)
        self.cv5 = nn.Conv2d(c_p5, 1)
        self.fusion = nn.Conv2d(out * 3, out, 1) # fast 1x1 conv

    def forward(self, x):
        p3, p4, p5 = torch.split(x, [self.c_p3, self.c_p4, self.c_p5], dim=1)
        cat = torch.cat([self.cv3(p3), self.cv4(p4), self.cv5(p5)], dim=1)
        return self.fusion(cat)

#heavy modules(Original DFEM-NEt)

class DeformableConv2d(nn.Module):
    def __init__(self, inc, outc, kernel_size=3, padding=1, stride=1, bias=None, modulation=True):
        super(DeformableConv2d, self).__init__()
        self.kernel_size = kernel_size
        self.padding = padding
        self.stride = stride
        self.offset_conv = nn.Conv2d(inc, 2 * kernel_size * kernel_size, kernel_size=3, padding=1, stride=stride)
        self.modulator_conv = nn.Conv2d(inc, kernel_size * kernel_size, kernel_size=3, padding=1, stride=stride)
        self.regular_conv = nn.Conv2d(inc, outc, kernel_size=kernel_size, padding=padding, stride=stride, bias=bias)
    def forward(self, x):
        offset = self.offset_conv(x)
        modulator = 2. * torch.sigmoid(self.modulator_conv(x))
        x = torchvision.ops.deform_conv2d(input=x, offset=offset, weight=self.regular_conv.weight, bias=regular.conv_bias, padding=self.padding, mask=modulator, stride=self.stride)
        return x

#true 3d fusion

class Scalseq_Heavy(nn.Module):
    def __init__(self, *args):
        super().__init__()
        flat_args=[]
        for a in args:
            if isinstance(a,(list, tuple)): flat_args.extend(a)
            else: flat_args.append(a)
        c_p3, c_p4, c_p5 = int(flat_args[-3]), int(flat_args[-2]), int(flat_args[-1])
        self.c_p3, self.c_p4, self.c_p5 = c_p3, c_p4, c_p5
        out = c_p3
        self.cv3 = nn.Cond2d(c_p3, out,1)
        self.cv4 = nn.Conv2d(c_p4, out,1)
        self.cv5 = nn.Conv2d(c_p5, out,1)
        self.fusion = nn.Conv3d(out, out,kernel_size=(3,1,1), bias = False) #slow 3d conv

    def forward(self, x):
        p3, p4, p5 = torch.split(x, [self.c_p3, self.c_p4, self.c_p5], dim=1)
        f3,f4,f5= self.cv3(p3), self.cv4(p4), self.cv5(p5)
        stack = torch.stack([f3,f4,f5], dim=2)
        out = self.fusion(stack)
        return out.squeeze(2)

#Wrapper block(Tua bottleneck)

class TuaBottleneck(nn.Module):
    def __init__(self, c1, c2, shortcut=True, e=0.5):
        super().__init__()
        c1,c2 = int(c1), int(c2)
        self.add = shortcut and c1==c2
        self.attn = DilatedConv(c1, c1, k=3, d=2)
        self.mlp = nn.Sequential(nn.Conv2d(c1, c2, 1), nn.Silu(), nn.Conv2d(c2, c2, 1))
    def forward(self, x):
        y = self.attn(x)
        return x + self.mlp(y) if self.add else self.mlp(y)

setattr(ultralytics.nn.tasks, 'TuaBottleneck', TuaBottleneck)
setattr(ultralytics.nn.tasks, 'Scalseq', Scalseq_Fast)
setattr(ultralytics.nn.tasks, 'Scalseq_Heavy', Scalseq_Heavy)
setattr(ultralytics.nn.tasks, 'DeformableConv2d', DialatedConv)
print("Architectures Registered")

Architectures Registered


In [3]:
%%writefile bhaskar_net.yaml
nc: 80
scales:
  n: [0.33, 0.25, 1024]
backbone:
  - [-1, 1, Conv, [64, 3, 2]]
  - [-1, 1, Conv, [128, 3, 2]]
  - [-1, 3, C2f, [128, True]]
  - [-1, 1, Conv, [256, 3, 2]]
  - [-1, 6, TuaBottleneck, [256, True]]
  - [-1, 1, Conv, [512, 3, 2]]
  - [-1, 6, TuaBottleneck, [512, True]]
  - [-1, 1, Conv, [1024, 3, 2]]
  - [-1, 3, C2f, [1024, True]]
  - [-1, 1, SPPF, [1024, 5]]
head:
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 6], 1, Concat, [1]]
  - [-1, 3, C2f, [512]]
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 4], 1, Concat, [1]]
  - [-1, 3, C2f, [256]]
  - [-1, 1, Conv, [256, 3, 2]]
  - [[-1, 12], 1, Concat, [1]]
  - [-1, 3, C2f, [512]]
  - [-1, 1, Conv, [512, 3, 2]]
  - [[-1, 9], 1, Concat, [1]]
  - [-1, 3, C2f, [1024]]
  - [[15, 18, 21], 1, Scalseq, [256, 512, 1024]] # FAST FUSION
  - [[22], 1, Detect, [nc]]

Writing bhaskar_net.yaml


In [4]:
%%writefile dfem_net_original.yaml
nc: 80
scales:
  n: [0.33, 0.25, 1024]
backbone:
  - [-1, 1, Conv, [64, 3, 2]]
  - [-1, 1, Conv, [128, 3, 2]]
  - [-1, 3, C2f, [128, True]]
  - [-1, 1, Conv, [256, 3, 2]]
  - [-1, 6, TuaBottleneck, [256, True]]
  - [-1, 1, Conv, [512, 3, 2]]
  - [-1, 6, TuaBottleneck, [512, True]]
  - [-1, 1, Conv, [1024, 3, 2]]
  - [-1, 3, C2f, [1024, True]]
  - [-1, 1, SPPF, [1024, 5]]
head:
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 6], 1, Concat, [1]]
  - [-1, 3, C2f, [512]]
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 4], 1, Concat, [1]]
  - [-1, 3, C2f, [256]]
  - [-1, 1, Conv, [256, 3, 2]]
  - [[-1, 12], 1, Concat, [1]]
  - [-1, 3, C2f, [512]]
  - [-1, 1, Conv, [512, 3, 2]]
  - [[-1, 9], 1, Concat, [1]]
  - [-1, 3, C2f, [1024]]
  - [[15, 18, 21], 1, Scalseq_Heavy, [256, 512, 1024]] # SLOW FUSION
  - [[22], 1, Detect, [nc]]


Writing dfem_net_original.yaml


In [ ]:
# --- FIND DATASETS AUTOMATICALLY ---
datasets = {"COCO128": "coco128.yaml"} # Default

# Try to find DAWN or Fog dataset
search_paths = glob.glob("/kaggle/input/**/data.yaml", recursive=True)
if search_paths:
    print(f"✅ Found External Dataset: {search_paths[0]}")
    datasets["Fog_Weather"] = search_paths[0]
else:
    print("⚠️ DAWN Dataset not found. Running only COCO128.")

# --- DEFINE MODELS ---
models = {
    "YOLOv8": "yolov8n.pt",
    "BhaskarNet": "bhaskar_net.yaml",
    "DFEM_Original": "dfem_net_original.yaml"
}

# --- RUN TOURNAMENT ---
for data_name, data_path in datasets.items():
    print(f"\n{'='*30}\nDATASET: {data_name}\n{'='*30}")
    
    for model_name, model_cfg in models.items():
        print(f"👉 Training {model_name}...")
        
        # 50 Epochs for Fast models, 10 for Slow (to finish in time)
        epochs = 10 if "Original" in model_name else 50
        
        try:
            model = YOLO(model_cfg)
            model.train(
                data=data_path, 
                epochs=epochs, 
                imgsz=640, 
                batch=16, 
                project='Research_Results',
                name=f"{model_name}_{data_name}",
                verbose=False # Keep output clean
            )
            print(f"✅ {model_name} Finished!")
        except Exception as e:
            print(f"❌ {model_name} Failed: {e}")

⚠️ DAWN Dataset not found. Running only COCO128.

DATASET: COCO128
👉 Training YOLOv8...
Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco128.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=YOLOv8_COCO128, nbs=64, nms=Fals

In [ ]:
# --- ZIP RESULTS ---
!zip -r Final_Research_Data.zip Research_Results

print("\n" + "="*40)
print("🎉 DONE! CLICK THE LINK BELOW TO DOWNLOAD")
print("="*40)

from IPython.display import FileLink
FileLink(r'Final_Research_Data.zip')